# load package

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer.rotator import Rotator

# 数据流详解

1. 输入数据的核心是5586行文本。  
2. 经过 embedding 之后，得到的是一个numpy矩阵，shape 是 (5586, 768)。这个 embeddings 矩阵保存在文件 text_embeddings_clean_lb2.npy 中。  
3. 再对这个 embedding 矩阵进行标准化操作， 得到 `scaled_embeddings`，可以将 `scaled_embeddings` 理解为所有5586个文本在768维度空间的坐标值的集合。  
4. 第一次PCA降维的目的是找到期望的维度数量，为第二次降维做准备。它得到的pca_result，是一个二维数组，形状为 `(n_samples:5586, n_components:34)`，表示每个样本在主成分空间中的得分。这个值在本段代码中没有后续使用。  
5. 第二次PCA降维，得到的一个名为 `pca` 的 object（对象，计算机术语），它的属性 `pca.components_` 是 PCA 模型计算得到的**主成分的特征向量**（又称为**载荷**或**主成分载荷**），其形状为 `(n_components, n_features)`，其中：
- - `n_components` 是保留的主成分数量，本例中是34。
- - `n_features` 是原始数据的特征数量（即每个样本的原始维度），本例中是768。
  - 本例中  `pca.components_` 的形状是: (34, 768)  

6. 对 `pca.components_` 进行转置，得到变量 `loadings`。  本例中它的形状是: (768, 34)
7. 再对变量 `loadings` 进行 Varimax rotation 操作，得到变量 `rotated_loadings` 。这个转换后得到的变量 `rotated_loadings` 与旧变量 `loadings` 形状相同，但是内部的值不同。 在本例中 `rotated_loadings`的形状是: (768, 34)
8. 通过 `rotated_scores = np.dot(scaled_embeddings, rotated_loadings)` 将5586个文本在768维度空间的坐标值，转换到，经过PCA降维和 Varimax rotation 之后，得到的**新坐标系**中的值。这些文本在新坐标系中的值的集合就是 `rotated_scores` , `rotated_scores` 的形状是: (5586, 34)

## 主成分的特征向量解读

在主成分分析（PCA）中，**主成分的特征向量**（也称为**载荷**或**主成分载荷**）是 PCA 模型计算的关键结果之一。它们表示数据在主成分空间中的方向，具体来说，特征向量描述了每个主成分如何由原始特征线性组合而成。

**特征向量的定义**  
- 特征向量是 PCA 模型计算的权重向量，形状为 (n_components, n_features)。
- 每个特征向量对应一个主成分，表示该主成分在原始特征空间中的方向。
- 特征向量的每个元素表示原始特征对该主成分的贡献程度。

**特征向量的作用**
1. **解释主成分：**
- - 特征向量可以帮助解释每个主成分与原始特征之间的关系。  
- - 例如，如果某个特征向量的某个元素值很大（接近 1 或 -1），说明该原始特征对该主成分的贡献很大。

2. **降维：**
- - 特征向量用于将数据从高维空间投影到低维的主成分空间。
- - 通过矩阵乘法，可以将标准化后的数据（scaled_embeddings）与特征向量相乘，得到降维后的数据。

3. **旋转：**
- - 在因子分析中，特征向量可以用于旋转（如 Varimax 旋转），以提高主成分的可解释性。 
 
**总结**
- **主成分的特征向量**是 PCA 模型计算的权重向量，表示每个主成分如何由原始特征线性组合而成。  
- 特征向量的形状为 **(n_components, n_features)**    
- 特征向量可以帮助解释主成分与原始特征之间的关系，并用于降维和旋转。  
- 在 **sklearn** 中，特征向量可以通过 `pca.components_` 属性获取。

# 代码段 `get the new score after varimax rotation` 解释

## 总结 - 这段代码的整个过程符合因子分析的标准流程。
- 这段代码得到的因子更加贴近 embedding 的768个原始维度，不像PCA得到的主成分是768个原始维度的正交线性组合。  
- 
这段代码的数学计算逻辑是正确的。它首先对数据进行标准化，然后通过PCA降维，接着使用Varimax旋转来增强因子的解释性，最后将数据投影到旋转后**的主成**分上并保存结果

**补充说明**  
1. varimax 旋转的目的是使得到的**主成分**更易于解释。为此，旋转后的因子载荷矩阵中，各个**主成分**会在某些**因子**上具有较高的载荷，而在其他因子上具有较低的载荷。
- 本例中，**因子**是 embedding 得到的768个维度。本例共有768个因子
- 本例中，得到的**主成分**是共有34个
 
2. 虽然与PCA分析部分代码相同，这段因子分析代码得到的仍然是34个新的**主成分**。但是这34个**主成分**的实质却是不同的，区别如下：
- PCA分析部分代码得到的34个**主成分** 中，每个主成分是 embedding 的768个维度的**线性组合**，这个**线性组合**的计算由**主成分的特征向量**（也称为**载荷**或**主成分载荷**）表达，本段代码中就是`pca.components_.`。
- - PCA得到的**主成分** 互相是**正交**的。  
- 这段因子分析代码得到的34个**主成分** 中，每个主成分也是 embedding 的768个维度的**线性组合**。本段代码中，这个**线性组合**的就是 `rotated_loadings` 。
- - 因子分析得到的**主成分** 互相不是正交的，而是会让：各个**主成分**会在某些**因子**上具有较高的载荷，而在其他因子上具有较低的载荷。从而提高可解释性。。

## 原代码  

In [11]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer.rotator import Rotator

explained_variance_ratio = 0.7   # 因为是截取的代码片段，所以需要设置这个参数   

# Step 1: Load the Data
embeddings_path = r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"
data_path = r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv"

embeddings = np.load(embeddings_path)
df = pd.read_csv(data_path)

# Step 2: Standardize the Embeddings
print("Standardizing embeddings...")
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

# Step 3: Perform PCA and Varimax Rotation
# Perform PCA to determine the number of components explaining 90% variance
pca = PCA()
pca_result = pca.fit_transform(scaled_embeddings)
cumulative_variance = pca.explained_variance_ratio_.cumsum()
threshold = explained_variance_ratio         # keep same as themes analysis
n_components = np.argmax(cumulative_variance >= threshold) + 1

# Fit PCA with the selected number of components
pca = PCA(n_components=n_components)
pca.fit(scaled_embeddings)

# Extract PCA loadings and transpose them
loadings = pca.components_.T

# Apply Varimax rotation
rotator = Rotator(method='varimax')
rotated_loadings = rotator.fit_transform(loadings)

# Step 4: Project Data onto Rotated Components
# Calculate the new scores by multiplying the standardized embeddings with the rotated loadings
rotated_scores = np.dot(scaled_embeddings, rotated_loadings)

# Step 5: Save the New Scores
# Save the rotated scores to a new file
output_path = r"V:\20240920\theme_analysis_act3301\rotated_scores.npy"
np.save(output_path, rotated_scores)

# Print the shape of the rotated scores
print(f"Rotated Scores' shape: {rotated_scores.shape}")

# Optional: Save the rotated scores with corresponding text data to a CSV file
rotated_scores_df = pd.DataFrame(rotated_scores, columns=[f"Rotated_Component_{i+1}" for i in range(n_components)])
rotated_scores_df["letter"] = df["letter"].values
rotated_scores_df.to_csv(r"V:\20240920\theme_analysis_act3301\rotated_scores_with_text.csv", index=False)

Standardizing embeddings...
Rotated Scores' shape: (5586, 34)


## 拆解代码，详细分析

### Step 1 & 2 代码分析

In [3]:
# Step 1: Load the Data
embeddings_path = r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"
data_path = r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv"

embeddings = np.load(embeddings_path)
df = pd.read_csv(data_path)

# Step 2: Standardize the Embeddings
print("Standardizing embeddings...")
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

Standardizing embeddings...


### Step 3 代码分析

**这里的代码逻辑与函数 `analyze_pca_themes` 存在不同：**
1. PCA计算次数不同：  
- 函数 `analyze_pca_themes` 在调用时，直接指定了降维的数量参数 `n_components`，只进行一次PCA计算
- 此处，根据参数 `explained_variance_ratio`，做第一次PCA计算，得到降维的数量参数 `n_components` ，然后根据第一次PCA计算得到的降维的数量参数 `n_components` ，再做第二次PCA计算。
2. PCA结果保存不同：  
- 函数 `analyze_pca_themes` 中，通过代码 `pca_result = pca.fit_transform(scaled_embeddings)`将PCA的结果中的**主成分得分**显式保存在名字为 `pca_result` 的二维数组（或矩阵）中，供后续使用。因为 `fit_transform` 返回降维后的数据，即**主成分得分**。
- 此处，`pca.fit(scaled_embeddings)` 则是将PCA得到的结果隐式保存在名为 pca 的 object（对象，计算机术语）的各个属性中。关键属性属性如下：
- - `pca.components_`：形状为 `(n_components, n_features)`，表示每个主成分的权重（即特征向量）。   
  - `pca.explained_variance_ratio_`：形状为 `(n_components,)`，表示每个主成分解释的方差比例。  
  - `pca.explained_variance_`：形状为 `(n_components,)`，表示每个主成分解释的方差值。  
  - `pca.mean_`：形状为 `(n_features,)`，表示数据的均值（在标准化后通常接近 0）。 

In [9]:
explained_variance_ratio = 0.7   # 为了理解代码添加，因为是截取的代码片段，所以需要设置这个参数  

# Step 3: Perform PCA and Varimax Rotation
# Perform PCA to determine the number of components explaining 90% variance
pca = PCA()
pca_result = pca.fit_transform(scaled_embeddings)      # fit_transform()得到降维后的数据，即**主成分得分**
cumulative_variance = pca.explained_variance_ratio_.cumsum()
threshold = explained_variance_ratio         # keep same as themes analysis
n_components = np.argmax(cumulative_variance >= threshold) + 1
print(f"\n --- The shape of 'pca_result' is: {pca_result.shape}")  # 为了理解代码添加, 行是文本数量， 列是原始 embeddings 的768个维度

# Fit PCA with the selected number of components
pca = PCA(n_components=n_components)            
pca.fit(scaled_embeddings)     #  在执行 `pca.fit()` 之后，PCA 模型的所有结果会保存在名为 `pca` 对象的各个属性中

print(f" --- The shape of 'pca.components_' is: {pca.components_.shape}")  # 为了理解代码添加，行是 principal component， 列是原始 embeddings 的768个维度
print(f" --- 注意 'pca_result' 和 'pca.components_' 两者的区别") 

# Extract PCA loadings and transpose them
loadings = pca.components_.T        # 为了满足后续 Varimax rotation 的输入格式要求: column corresponds to feature (principal component)
print(f"\n --- The shape of loadings is: {loadings.shape}")     # 为了理解代码添加

# Apply Varimax rotation
rotator = Rotator(method='varimax')
rotated_loadings = rotator.fit_transform(loadings)
print(f"\n --- The shape of rotated_loadings is: {rotated_loadings.shape}")     # 为了理解代码添加


 --- The shape of 'pca_result' is: (5586, 768)
 --- The shape of 'pca.components_' is: (34, 768)
 --- 注意 'pca_result' 和 'pca.components_' 两者的区别

 --- The shape of loadings is: (768, 34)

 --- The shape of rotated_loadings is: (768, 34)


<font color=darkred size=3.5>  注释  </font>  
  
**`pca.components_` 是 PCA 模型计算的主成分（特征向量）**  
`pca.components_` 是 PCA 模型计算的主成分（特征向量），其形状为 `(n_components, n_features)`，其中：  
- `n_components` 是保留的主成分数量。  
- `n_features` 是原始数据的特征数量（即每个样本的维度）。  

**`pca_result = pca.fit_transform(scaled_embeddings)`**   
- `pca_result`是返回的降维后的数据，不是 PCA 模型对象。  
- 返回值是一个二维数组，形状为 `(n_samples, n_components)`，表示每个样本在主成分空间中的得分。

#### 辅助理解的代码

In [28]:
from sklearn.decomposition import PCA
import numpy as np

# 示例数据
demo_scaled_embeddings = np.random.rand(1000, 100)  # 1000 个样本，每个样本有 100 个特征

# 初始化并拟合 PCA 模型
pca = PCA(n_components=5)
pca.fit(demo_scaled_embeddings)

# 列出 PCA 对象的所有属性和方法
print(dir(pca))

# 查看主成分
print("\n ---主成分 (components_) 的 shape:", pca.components_.shape)

# 查看解释的方差比例
print("\n ---解释的方差比例 (explained_variance_ratio_) 的 shape:", pca.explained_variance_ratio_.shape)

# 查看奇异值
print("\n ---奇异值 (singular_values_) 的 shape:", pca.singular_values_.shape)

# 查看数据的均值
print("\n ---数据的均值 (mean_) 的 shape:", pca.mean_.shape)

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_build_request_for_signature', '_check_feature_names', '_check_n_features', '_doc_link_module', '_doc_link_template', '_doc_link_url_param_generator', '_fit', '_fit_full', '_fit_svd_solver', '_fit_truncated', '_get_default_requests', '_get_doc_link', '_get_metadata_request', '_get_param_names', '_get_tags', '_more_tags', '_n_features_out', '_parameter_constraints', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_sklearn_auto_wrap_output_keys', '_validate_data', '_validate_params', 'components_', 'copy', 'explained_variance_', 'exp

In [29]:
from sklearn.decomposition import PCA
from factor_analyzer.rotator import Rotator
import numpy as np

# 示例数据
demo_scaled_embeddings = np.random.rand(1000, 100)  # 1000 个样本，每个样本有 100 个特征

# 初始化并拟合 PCA 模型
pca = PCA(n_components=5)
pca.fit(demo_scaled_embeddings)

# 获取主成分并转置
demo_loadings = pca.components_.T  # 形状为 (100, 5)

# 初始化 Rotator 并执行 Varimax 旋转
rotator = Rotator(method='varimax')
demo_rotated_loadings = rotator.fit_transform(demo_loadings)

# 打印旋转后的主成分
# print("旋转后的主成分 (rotated_loadings):", rotated_loadings)
print("旋转后的主成分形状:", demo_rotated_loadings.shape)  # 输出: (100, 5)

旋转后的主成分形状: (100, 5)


### Step 4 & 5 代码分析

In [10]:
# Step 4: Project Data onto Rotated Components
# Calculate the new scores by multiplying the standardized embeddings with the rotated loadings
rotated_scores = np.dot(scaled_embeddings, rotated_loadings)

# Step 5: Save the New Scores
# Save the rotated scores to a new file
output_path = r"V:\20240920\theme_analysis_act3301\rotated_scores.npy"
np.save(output_path, rotated_scores)

# Print the shape of the rotated scores
print(f"`Rotated Scores' shape: {rotated_scores.shape}")


`Rotated Scores' shape: (5586, 34)


### 构建并保存文件部分的代码解析

In [ ]:
# Optional: Save the rotated scores with corresponding text data to a CSV file
# 用 rotated_scores 构建一个新的 DataFrame，并为它的每个列分配一个有意义的列名,即 "Rotated_Component_i"
rotated_scores_df = pd.DataFrame(rotated_scores, columns=[f"Rotated_Component_{i+1}" for i in range(n_components)])

# 为 rotated_scores_df 增加列 'letter',内容是 df["letter"].values，也就是csv文件的 letter 列
# 这样就得到了一个 datframe，它包含每个letter 以及它在经过varimax旋转的坐标轴上的得分
rotated_scores_df["letter"] = df["letter"].values
rotated_scores_df.to_csv(r"V:\20240920\theme_analysis_act3301\rotated_scores_with_text.csv", index=False)

#### 辅助理解的代码

In [37]:
columns=[f"Rotated_Component_{i+1}" for i in range(n_components)]
columns

['Rotated_Component_1',
 'Rotated_Component_2',
 'Rotated_Component_3',
 'Rotated_Component_4',
 'Rotated_Component_5',
 'Rotated_Component_6',
 'Rotated_Component_7',
 'Rotated_Component_8',
 'Rotated_Component_9',
 'Rotated_Component_10',
 'Rotated_Component_11',
 'Rotated_Component_12',
 'Rotated_Component_13',
 'Rotated_Component_14',
 'Rotated_Component_15',
 'Rotated_Component_16',
 'Rotated_Component_17',
 'Rotated_Component_18',
 'Rotated_Component_19',
 'Rotated_Component_20',
 'Rotated_Component_21',
 'Rotated_Component_22',
 'Rotated_Component_23',
 'Rotated_Component_24',
 'Rotated_Component_25',
 'Rotated_Component_26',
 'Rotated_Component_27',
 'Rotated_Component_28',
 'Rotated_Component_29',
 'Rotated_Component_30',
 'Rotated_Component_31',
 'Rotated_Component_32',
 'Rotated_Component_33',
 'Rotated_Component_34']

In [38]:
rotated_scores_df.head(2)

,Rotated_Component_1,Rotated_Component_2,Rotated_Component_3,Rotated_Component_4,Rotated_Component_5,Rotated_Component_6,Rotated_Component_7,Rotated_Component_8,Rotated_Component_9,Rotated_Component_10,...,Rotated_Component_26,Rotated_Component_27,Rotated_Component_28,Rotated_Component_29,Rotated_Component_30,Rotated_Component_31,Rotated_Component_32,Rotated_Component_33,Rotated_Component_34,letter
0,9.167921,1.792850,-0.556115,-1.590027,6.060188,2.637957,11.487621,-1.190416,0.123043,3.591494,...,0.98753,3.833739,2.013072,-2.597245,2.215535,1.577757,6.106043,5.121364,-3.858929,!!! requestor: this is entirely too much writi...
1,13.461669,-1.829778,-1.323529,-11.879486,-6.598827,2.314043,-1.849994,11.593370,2.812556,-4.729143,...,4.94258,-11.247339,2.403609,0.044364,2.285817,-3.689269,-5.199858,2.533543,9.381222,. this office would work with other department...


In [ ]:
<font color=darkred size=3.5>    </font>

# 代码段 `get top_n positive and negative score letter of each Rotated Component` 解析

## 原代码  

In [35]:
import pandas as pd

# Define the number of top scores to extract
top_n = 5

# Extract Top `top_n` Positive and Negative Scores for Each Rotated Component
results = []
for component in rotated_scores_df.columns[:-1]:  # Exclude the "letter" column
    # Get the top_n positive scores
    top_positive = rotated_scores_df.nlargest(top_n, component)[["letter", component]]
    
    # Get the top_n negative scores
    top_negative = rotated_scores_df.nsmallest(top_n, component)[["letter", component]]
    
    # Store the results for positive scores
    for letter, score in top_positive.values:
        results.append({
            "Component": component,
            "Type": "Positive",
            "Letter": letter,
            "Score": score
        })
    
    # Store the results for negative scores
    for letter, score in top_negative.values:
        results.append({
            "Component": component,
            "Type": "Negative",
            "Letter": letter,
            "Score": score
        })

# Step 3: Save the Results to a CSV File
# Create a DataFrame to store the results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
output_path = r"V:\20240920\theme_analysis_act3301\top_scores_per_rotated_component.csv"
results_df.to_csv(output_path, index=False)

# Print the results
print(results_df)

                Component      Type  \
0     Rotated_Component_1  Positive   
1     Rotated_Component_1  Positive   
2     Rotated_Component_1  Positive   
3     Rotated_Component_1  Positive   
4     Rotated_Component_1  Positive   
..                    ...       ...   
335  Rotated_Component_34  Negative   
336  Rotated_Component_34  Negative   
337  Rotated_Component_34  Negative   
338  Rotated_Component_34  Negative   
339  Rotated_Component_34  Negative   

                                                Letter      Score  
0    equality means being treated the same. special...  21.030592  
1    i the world needs balance. the rich and powerf...  18.291817  
2    everyone should have equal access to learning ...  17.791910  
3    the inequity in america is absolutely abhorren...  17.730347  
4    this is a women‚äôs global empowerment, develo...  17.642102  
..                                                 ...        ...  
335  please reintroduce act3301. it is an important... 

## 拆解代码，详细分析

### 取dataframe的列名，并排除指定的列名

#### 原代码

通过 letter 列的位置排除它的列名

In [36]:
rotated_scores_df.columns

Index(['Rotated_Component_1', 'Rotated_Component_2', 'Rotated_Component_3',
       'Rotated_Component_4', 'Rotated_Component_5', 'Rotated_Component_6',
       'Rotated_Component_7', 'Rotated_Component_8', 'Rotated_Component_9',
       'Rotated_Component_10', 'Rotated_Component_11', 'Rotated_Component_12',
       'Rotated_Component_13', 'Rotated_Component_14', 'Rotated_Component_15',
       'Rotated_Component_16', 'Rotated_Component_17', 'Rotated_Component_18',
       'Rotated_Component_19', 'Rotated_Component_20', 'Rotated_Component_21',
       'Rotated_Component_22', 'Rotated_Component_23', 'Rotated_Component_24',
       'Rotated_Component_25', 'Rotated_Component_26', 'Rotated_Component_27',
       'Rotated_Component_28', 'Rotated_Component_29', 'Rotated_Component_30',
       'Rotated_Component_31', 'Rotated_Component_32', 'Rotated_Component_33',
       'Rotated_Component_34', 'letter'],
      dtype='object')

In [ ]:
rotated_scores_df.columns[:-1] # Exclude the "letter" column

#### 我的改进建议

In [37]:
# 指定要排除的列名
exclude_columns = {'letter'}

# 使用 difference 方法排除指定列
rotated_scores_df.columns.difference(exclude_columns)


Index(['Rotated_Component_1', 'Rotated_Component_10', 'Rotated_Component_11',
       'Rotated_Component_12', 'Rotated_Component_13', 'Rotated_Component_14',
       'Rotated_Component_15', 'Rotated_Component_16', 'Rotated_Component_17',
       'Rotated_Component_18', 'Rotated_Component_19', 'Rotated_Component_2',
       'Rotated_Component_20', 'Rotated_Component_21', 'Rotated_Component_22',
       'Rotated_Component_23', 'Rotated_Component_24', 'Rotated_Component_25',
       'Rotated_Component_26', 'Rotated_Component_27', 'Rotated_Component_28',
       'Rotated_Component_29', 'Rotated_Component_3', 'Rotated_Component_30',
       'Rotated_Component_31', 'Rotated_Component_32', 'Rotated_Component_33',
       'Rotated_Component_34', 'Rotated_Component_4', 'Rotated_Component_5',
       'Rotated_Component_6', 'Rotated_Component_7', 'Rotated_Component_8',
       'Rotated_Component_9'],
      dtype='object')

### 取 Rotated_Component_6 中最大的 top_n 个 score 对应的letter

**运行逻辑**
- 从 `rotated_scores_df` 中取参数 `component` 指定的列数据
- 用 `nlargest` 排序，得到 `top_n` 个最大的值所在**行的索引值**
- 然后用这些**行的索引值**找到这些行，取出这些行中列名为 `["letter", component]` 的列中的值
- 最后用这些取到的值构建 dataframe `top_positive`

In [38]:
top_n = 5
component = 'Rotated_Component_6'

# 取 Rotated_Component_6 中最大的 top_n 个 score 对应的letter
top_positive = rotated_scores_df.nlargest(top_n, component)[["letter", component]]
top_positive

,letter,Rotated_Component_6
4139,please re-introduce the bill to help the effor...,14.873634
3877,leadership is displayed in various ways and ac...,14.690495
4108,please help re-introduce the bill and encourag...,14.280506
2630,i believe that the bill should be re-introduce...,14.213353
3098,i impose that the bill regarding women's empow...,13.893262


### 取 Rotated_Component_6 中最小的 top_n 个 score 对应的letter

**运行逻辑**
- 从 `rotated_scores_df` 中取参数 `component` 指定的列数据
- 用 `nsmallest` 排序，得到 `top_n` 个最小的值所在**行的索引值**
- 然后用这些**行的索引值**找到这些行，取出这些行中列名为 `["letter", component]` 的列中的值
- 最后用这些取到的值构建 dataframe `top_negative`

In [39]:
# Get the top_n negative scores
top_negative = rotated_scores_df.nsmallest(top_n, component)[["letter", component]]
top_negative

,letter,Rotated_Component_6
1840,empowerment of women in the us is not in the p...,-14.876619
2805,i do not feel act3301 is neccesary. it seems l...,-14.199014
4938,"act3301 is regarding a non-issue. also, it pro...",-14.090655
4847,the act3301 is nothing but the creation of ano...,-14.081499
2912,i don't think act3301 is necessary. it is too ...,-13.743364


### 构建 list `results`

In [44]:
# Extract Top `top_n` Positive and Negative Scores for Each Rotated Component
results = []
for component in rotated_scores_df.columns[:-1]:  # Exclude the "letter" column
    # Get the top_n positive scores
    top_positive = rotated_scores_df.nlargest(top_n, component)[["letter", component]]
    
    # Get the top_n negative scores
    top_negative = rotated_scores_df.nsmallest(top_n, component)[["letter", component]]
    
    # Store the results for positive scores
    for letter, score in top_positive.values:
        results.append({
            "Component": component,
            "Type": "Positive",
            "Letter": letter,
            "Score": score
        })
    
    # Store the results for negative scores
    for letter, score in top_negative.values:
        results.append({
            "Component": component,
            "Type": "Negative",
            "Letter": letter,
            "Score": score
        })

# 显示变量 results 的前n个元素
print(f"\n --- results 的元素数据类型是：{type(results[1])}  ")

n = 7
print(f"\n --- results 的前{n}个元素数据的值是：  ")
results[:n]


 --- results 的元素数据类型是：<class 'dict'>  

 --- results 的前7个元素数据的值是：  


[{'Component': 'Rotated_Component_1',
  'Type': 'Positive',
  'Letter': 'equality means being treated the same. special treatment is not equality.',
  'Score': 21.03059171997572},
 {'Component': 'Rotated_Component_1',
  'Type': 'Positive',
  'Letter': 'i the world needs balance. the rich and powerful have looked out for themselves for too long.',
  'Score': 18.291817315577514},
 {'Component': 'Rotated_Component_1',
  'Type': 'Positive',
  'Letter': 'everyone should have equal access to learning and job opportunities for leadership.',
  'Score': 17.791909845440784},
 {'Component': 'Rotated_Component_1',
  'Type': 'Positive',
  'Letter': 'the inequity in america is absolutely abhorrent on all levels, be it age, race, gender, or sexuality. as someone in a position of power, i urge to you to help decrease the disparity between men and women and close the income gap. we have to get other people in positions of power to increase the number of viewpoints and change america for the better',
  

### 用list `results` 构建 dataframe `results_df`

In [46]:
# Create a DataFrame to store the results
results_df = pd.DataFrame(results)

n = 7
print(f"\n --- DataFrame 的前{n}行数据的值是：  ")
results_df.head(n)


 --- DataFrame 的前7行数据的值是：  


,Component,Type,Letter,Score
0,Rotated_Component_1,Positive,equality means being treated the same. special...,21.030592
1,Rotated_Component_1,Positive,i the world needs balance. the rich and powerf...,18.291817
2,Rotated_Component_1,Positive,everyone should have equal access to learning ...,17.791910
3,Rotated_Component_1,Positive,the inequity in america is absolutely abhorren...,17.730347
4,Rotated_Component_1,Positive,"this is a women‚äôs global empowerment, develo...",17.642102
5,Rotated_Component_1,Negative,"dear senator,\n i don't think we should reintr...",-14.264280
6,Rotated_Component_1,Negative,"dear senator,\n \n i am writing to encourage y...",-14.146990
